In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

In [ ]:
df_energy = pd.read_csv("/Users/qingshuzhao/Documents/ML_PPI_project/training_dataset/training_set_3_11/top_504_iforest_dropout.csv")
df_energy = df_energy.dropna().reset_index(drop=True)
# Add experimental binding energy
df_energy['dG_exp'] = (1.98722*298.15*np.log(df_energy['kd_molar'])/1000)
y_train=df_energy['dG_exp'].values
print(y_train)

In [4]:
df_PDBind = pd.read_csv("/Users/qingshuzhao/Documents/ML_PPI_project/training_results/top_504_PDBs_20250309_2241/PDBind_comparison.csv")
df_nanobody = pd.read_csv("/Users/qingshuzhao/Documents/ML_PPI_project/training_results/top_504_PDBs_20250309_2241/Nanobody_comparison.csv")
df_Nazmul = pd.read_csv("/Users/qingshuzhao/Documents/ML_PPI_project/training_results/top_504_PDBs_20250309_2241/Nazmul_comparison.csv")

In [5]:
exp_PDBind = df_PDBind['REAL'].values
exp_nanobody = df_nanobody['REAL'].values
exp_Nazmul = df_Nazmul['REAL'].values
print(exp_PDBind)

[-10.5   -8.31 -11.92  -8.95  -9.55 -10.85 -10.89 -13.06 -11.31  -7.05
  -9.   -14.08 -13.14 -14.58 -10.2   -9.    -7.18 -11.8   -7.8  -10.16
 -10.59 -10.64  -8.37  -8.95  -9.42 -14.02  -6.78 -10.25  -9.27 -10.97
 -10.1  -11.58 -12.07 -10.08  -8.09  -9.99 -10.27  -9.73  -8.27 -11.52
  -8.6   -7.95  -8.32  -8.19 -12.11]


In [6]:
pred_PDBind = df_PDBind['ANN_PREDICTED'].values
pred_nanobody = df_nanobody['ANN_PREDICTED'].values
pred_Nazmul = df_Nazmul['ANN_PREDICTED'].values
print(pred_PDBind)

[ -9.611544   -9.540731  -10.007715  -10.139776  -10.080409   -9.681791
  -9.960272   -9.71303    -9.610195  -10.019461   -9.558855   -9.711616
  -9.7540655  -9.641465   -9.73293    -9.837041   -9.827623  -10.088225
  -9.640346   -9.618047   -9.595985   -9.681526   -9.638271   -9.69504
  -9.742564   -9.987263   -9.700088   -9.5851     -9.6343355  -9.609354
  -9.630075   -9.5655575  -9.613466   -9.663034   -9.741429   -9.564753
  -9.54965    -9.578879   -9.745567   -9.759857   -9.836648   -9.906647
  -9.910566   -9.998722   -9.928999 ]


# mean

In [7]:
# Compute the mean of the training target
train_target_mean = np.mean(y_train)

# Define test sets and predictions
test_sets = {
    "PDBind": exp_PDBind,
    "Nanobody": exp_nanobody,
    "Nazmul": exp_Nazmul
}

predictions = {
    "PDBind": pred_PDBind,
    "Nanobody": pred_nanobody,
    "Nazmul": pred_Nazmul
}

# Loop over each test set and compare RMSE
for name in test_sets:
    y_test = test_sets[name]
    y_pred = predictions[name]

    # Create baseline predictions (training mean)
    baseline_predictions = np.full_like(y_test, train_target_mean)

    # Compute RMSE
    rmse_model = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_baseline = np.sqrt(mean_squared_error(y_test, baseline_predictions))

    # Print results
    print(f"\n--- {name} Test Set ---")
    print(f"Model RMSE: {rmse_model:.4f}")
    print(f"Baseline RMSE: {rmse_baseline:.4f}")

    # Check model performance
    if rmse_model < rmse_baseline:
        print(f"Model performs better than baseline on {name}.")
    else:
        print(f"Model performs worse than baseline on {name}. Features might not be informative.")


--- PDBind Test Set ---
Model RMSE: 1.9385
Baseline RMSE: 1.9491
Model performs better than baseline on PDBind.

--- Nanobody Test Set ---
Model RMSE: 2.3825
Baseline RMSE: 2.5585
Model performs better than baseline on Nanobody.

--- Nazmul Test Set ---
Model RMSE: 3.7492
Baseline RMSE: 3.8090
Model performs better than baseline on Nazmul.


# Linear 

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize the Linear Regression model
baseline_model = LinearRegression()

# Train using multiple features
X_train = df_energy.drop(columns=['dG_exp', 'pdb_id', 'resolution', 'kd_molar'], axis=1) 
y_train = df_energy['dG_exp'].values
baseline_model.fit(X_train, y_train)

# Load test sets
nanobody_data = pd.read_csv("/Users/qingshuzhao/Documents/ML_PPI_project/Paper/SI/validation_set/nanobody_testset.csv")
pdbind_data = pd.read_csv("/Users/qingshuzhao/Documents/ML_PPI_project/Paper/SI/validation_set/PDBind_testset.csv")
nazmul_data = pd.read_csv("/Users/qingshuzhao/Documents/ML_PPI_project/Paper/SI/validation_set/Nazmul_20_testset.csv")

# Prepare test features
X_test_PDBind = pdbind_data.drop(columns=['experimental', 'packstat', 'SCORE:', 'yhh_planarity', 'description'], axis=1)
X_test_nanobody = nanobody_data.drop(columns=['experimental', 'packstat', 'yhh_planarity', 'description'], axis=1)
X_test_Nazmul = nazmul_data.drop(columns=['experimental', 'packstat', 'yhh_planarity', 'description'], axis=1)

# Store test sets and expected values
test_sets = {
    "PDBind": (X_test_PDBind, exp_PDBind),
    "Nanobody": (X_test_nanobody, exp_nanobody),
    "Nazmul": (X_test_Nazmul, exp_Nazmul)
}

# Assume `predictions` contains neural network model predictions for each test set
predictions = {
    "PDBind": pred_PDBind,
    "Nanobody": pred_nanobody,
    "Nazmul": pred_Nazmul
}

# Compare neural network vs regression baseline
for name, (X_test, y_test) in test_sets.items():
    y_pred_regression = baseline_model.predict(X_test)
    y_pred_nn = predictions[name]

    # Compute RMSE
    rmse_regression = np.sqrt(mean_squared_error(y_test, y_pred_regression))
    rmse_nn = np.sqrt(mean_squared_error(y_test, y_pred_nn))

    # Print results
    print(f"\n--- {name} Test Set ---")
    print(f"Neural Network Model RMSE: {rmse_nn:.4f}")
    print(f"Linear Regression Baseline RMSE: {rmse_regression:.4f}")

    # Compare performance
    if rmse_nn < rmse_regression:
        print(f"Neural Network model outperforms Linear Regression on {name}.")
    else:
        print(f"Linear Regression outperforms Neural Network on {name}.")


--- PDBind Test Set ---
Neural Network Model RMSE: 1.9385
Linear Regression Baseline RMSE: 168.6976
Neural Network model outperforms Linear Regression on PDBind.

--- Nanobody Test Set ---
Neural Network Model RMSE: 2.3825
Linear Regression Baseline RMSE: 219.6221
Neural Network model outperforms Linear Regression on Nanobody.

--- Nazmul Test Set ---
Neural Network Model RMSE: 3.7492
Linear Regression Baseline RMSE: 40.7329
Neural Network model outperforms Linear Regression on Nazmul.
